<a href="https://colab.research.google.com/github/norahyk/pythonsnippet/blob/main/api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import json

# 祝日を取得

In [ ]:
url_fmt = 'https://holidays-jp.github.io/api/v1/{year}/date.json'
year = '2021'
res = requests.get(url_fmt.format(year=year))
holidays = json.loads(res.content)
holidays

{'2021-01-01': '元日',
 '2021-01-11': '成人の日',
 '2021-02-11': '建国記念の日',
 '2021-02-23': '天皇誕生日',
 '2021-03-20': '春分の日',
 '2021-04-29': '昭和の日',
 '2021-05-03': '憲法記念日',
 '2021-05-04': 'みどりの日',
 '2021-05-05': 'こどもの日',
 '2021-07-22': '海の日',
 '2021-07-23': 'スポーツの日',
 '2021-08-08': '山の日',
 '2021-08-09': '休日 山の日',
 '2021-09-20': '敬老の日',
 '2021-09-23': '秋分の日',
 '2021-11-03': '文化の日',
 '2021-11-23': '勤労感謝の日'}